In [9]:
!pip install tweepy nltk numpy pyspark

In [1]:
import tweepy, json, nltk
from nltk.corpus import twitter_samples, stopwords
import random
import numpy as np
import pandas as pd
import math

In [2]:
nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /home/shervin/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/shervin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')



In [4]:
# 80 % for trainset 20% for testset
all_tweets = all_positive_tweets[0:4000]+all_negative_tweets[0:4000]

In [5]:
def clean_the_tweet(tweet):
    words = tweet.split()
    res=[]
    for word in words:
        if ( word[0] not in ['@',':'] + stopwords.words('english')) and (not word.startswith('http://')) and (not word.startswith('https://')):
            word = word.replace('#','')
            word = word.replace('.','')
            word = word.lower()
            res.append(word)
    return list(set(res))

In [6]:
clean_the_tweet(all_tweets[0])

['community', 'engaged', 'followfriday', 'week', 'being', 'for']

In [7]:
import pyspark

In [8]:
spark = pyspark.sql.SparkSession.builder.master('local').appName('APP').getOrCreate()

22/07/31 18:07:17 WARN Utils: Your hostname, shervin resolves to a loopback address: 127.0.1.1; using 192.168.1.33 instead (on interface wlp2s0)
22/07/31 18:07:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/07/31 18:07:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
sc = spark.sparkContext

In [10]:
total = len(all_tweets)
total

8000

In [11]:
def make_the_pairs(tweet):
    tokens = clean_the_tweet(tweet)
    res = []
    for w1 in tokens:
        seen_tokens = []
        for w2 in tokens:
            if w1 != w2:
                res.append((w1,w2))
    return res



In [12]:
text_file = sc.parallelize(all_tweets)
counts = text_file.flatMap(clean_the_tweet) \
                            .map(lambda word: (word, 1)) \
                           .reduceByKey(lambda x, y: x + y)
c_out = counts.collect()
len(c_out)

10379

In [14]:
df_count = pd.DataFrame(c_out,columns=['WORD','COUNT'])
df_count.set_index('WORD',inplace=True)
df_count
df_count.to_csv('WORD_COUNT.csv')
df_count

,COUNT
WORD,
being,81
engaged,7
for,900
followfriday,23
week,85
...,...
unfollower,1
bianca,1
bun,1


In [15]:
t1 = text_file.flatMap(make_the_pairs) \
                .map(lambda x: (x,1)) \
                .reduceByKey(lambda a,b: a+b)
t1_out = t1.collect()
len(t1_out)

216494

In [16]:
len(t1_out)

216494

In [18]:
temp = t1.map(lambda x: ((x[0][0],x[0][1]),x[1]))
output = temp.collect()
len(output)

216494

In [19]:
df = pd.DataFrame(output,columns = ['RULE','SUPPORT'])
df['CONFIDENCE'] = df['SUPPORT']
df['SUPPORT'] /= total*(1/100)
count_column = []
for rule in df.RULE:
    count_column.append(df_count.loc[rule[0]].COUNT)
df['CONFIDENCE'] = df['CONFIDENCE'] / count_column
df

,RULE,SUPPORT,CONFIDENCE
0,"(being, engaged)",0.0750,0.074074
1,"(being, for)",0.4750,0.469136
2,"(being, followfriday)",0.2875,0.283951
3,"(being, week)",0.3125,0.308642
4,"(being, community)",0.2875,0.283951
...,...,...,...
216489,"(thursday, want)",0.0125,0.500000
216490,"(thursday, got)",0.0125,0.500000
216491,"(got, rang)",0.0125,0.007246
216492,"(got, holiday)",0.0125,0.007246


In [20]:
negative_rules_count = sc.parallelize(all_negative_tweets[0:4000]).flatMap(make_the_pairs) \
                .map(lambda x: (x,1)) \
                .reduceByKey(lambda a,b: a+b)
neg_out = negative_rules_count.collect()
len(neg_out)

112704

In [21]:
negative_rules = pd.DataFrame(neg_out, columns=['RULE','NEG_COUNT'])
negative_rules

,RULE,NEG_COUNT
0,"(for, hopeless)",1
1,"(hopeless, for)",1
2,"(cute, kids)",1
3,"(cute, ikea)",1
4,"(cute, 2)",1
...,...,...
112699,"(thursday, want)",1
112700,"(thursday, got)",1
112701,"(got, rang)",1
112702,"(got, holiday)",1


In [22]:
positive_rules_count = sc.parallelize(all_positive_tweets[0:4000]).flatMap(make_the_pairs) \
                .map(lambda x: (x,1)) \
                .reduceByKey(lambda a,b: a+b)
pos_out = positive_rules_count.collect()
len(pos_out)

116326

In [23]:
positive_rules = pd.DataFrame(pos_out,columns=['RULE','POS_COUNT'])
positive_rules

,RULE,POS_COUNT
0,"(being, engaged)",6
1,"(being, for)",35
2,"(being, followfriday)",23
3,"(being, week)",25
4,"(being, community)",23
...,...,...
116321,"(less, be)",1
116322,"(less, real,)",1
116323,"(less, painful)",1
116324,"(less, pain)",1


In [24]:
df = pd.merge(df,positive_rules,how='outer',on='RULE')
df = pd.merge(df,negative_rules,how='outer',on='RULE')

In [25]:
df['NEG_COUNT']=df['NEG_COUNT'].fillna(0)
df['POS_COUNT']=df['POS_COUNT'].fillna(0)
df

,RULE,SUPPORT,CONFIDENCE,POS_COUNT,NEG_COUNT
0,"(being, engaged)",0.0750,0.074074,6.0,0.0
1,"(being, for)",0.4750,0.469136,35.0,3.0
2,"(being, followfriday)",0.2875,0.283951,23.0,0.0
3,"(being, week)",0.3125,0.308642,25.0,0.0
4,"(being, community)",0.2875,0.283951,23.0,0.0
...,...,...,...,...,...
216489,"(thursday, want)",0.0125,0.500000,0.0,1.0
216490,"(thursday, got)",0.0125,0.500000,0.0,1.0
216491,"(got, rang)",0.0125,0.007246,0.0,1.0
216492,"(got, holiday)",0.0125,0.007246,0.0,1.0


In [379]:
df[ df['NEG_COUNT'] - df['POS_COUNT'] > 5]

,RULE,SUPPORT,CONFIDENCE,POS_COUNT,NEG_COUNT
43,"(we, last)",0.10,0.028490,1.0,9.0
53,"(last, we)",0.10,0.088496,1.0,9.0
135,"(just, but)",0.44,0.104762,13.0,31.0
137,"(for, not)",0.49,0.044104,16.0,33.0
140,"(not, for)",0.49,0.117506,16.0,33.0
...,...,...,...,...,...
249680,"(been, always)",0.06,0.037037,0.0,6.0
249883,"(-, indiemusic)",0.06,0.027397,0.0,6.0
250906,"(sexy, wet)",0.06,0.545455,0.0,6.0
250911,"(wet, snapchat)",0.06,0.461538,0.0,6.0


In [391]:
(np.absolute(df['POS_COUNT']-df['NEG_COUNT'])).mean()

1.2460428832254462

In [26]:
df['LABEL'] = np.absolute(df['POS_COUNT']-df['NEG_COUNT']).map(lambda x: 'POS' if x>= 0 else 'NEG' )
df

,RULE,SUPPORT,CONFIDENCE,POS_COUNT,NEG_COUNT,LABEL
0,"(being, engaged)",0.0750,0.074074,6.0,0.0,POS
1,"(being, for)",0.4750,0.469136,35.0,3.0,POS
2,"(being, followfriday)",0.2875,0.283951,23.0,0.0,POS
3,"(being, week)",0.3125,0.308642,25.0,0.0,POS
4,"(being, community)",0.2875,0.283951,23.0,0.0,POS
...,...,...,...,...,...,...
216489,"(thursday, want)",0.0125,0.500000,0.0,1.0,POS
216490,"(thursday, got)",0.0125,0.500000,0.0,1.0,POS
216491,"(got, rang)",0.0125,0.007246,0.0,1.0,POS
216492,"(got, holiday)",0.0125,0.007246,0.0,1.0,POS


In [29]:
df

,RULE,SUPPORT,CONFIDENCE,POS_COUNT,NEG_COUNT,LABEL
0,"(being, engaged)",0.0750,0.074074,6.0,0.0,POS
1,"(being, for)",0.4750,0.469136,35.0,3.0,POS
2,"(being, followfriday)",0.2875,0.283951,23.0,0.0,POS
3,"(being, week)",0.3125,0.308642,25.0,0.0,POS
4,"(being, community)",0.2875,0.283951,23.0,0.0,POS
...,...,...,...,...,...,...
216489,"(thursday, want)",0.0125,0.500000,0.0,1.0,POS
216490,"(thursday, got)",0.0125,0.500000,0.0,1.0,POS
216491,"(got, rang)",0.0125,0.007246,0.0,1.0,POS
216492,"(got, holiday)",0.0125,0.007246,0.0,1.0,POS


In [30]:
df.to_csv('RULE_SC.csv')

In [507]:
queries = []
actual = []
predicted = []
metrics = []

correct_answers = 0


for query in all_positive_tweets[4000:]:
    print(f' {query} of class POSITIVE:')
    queries.append(query)
    actual.append('POSITIVE')
    
    q_rules = sc.parallelize([query]).flatMap(make_the_pairs) \
                    .map(lambda x: (x,1)) \
                    .reduceByKey(lambda a,b: a+b)
    q_rules = q_rules.collect()

    #metric = (+ -)(support)*confident
    metric = 0
    for rule in q_rules:
        if len(df[df['RULE'] == rule[0]])>0:
            row = df[df['RULE'] == rule[0]]
            support = row.SUPPORT.values[0]
            confidence = row.CONFIDENCE.values[0]
            coef = row.POS_COUNT.values[0] - row.NEG_COUNT.values[0]
            metric += coef * support * confidence
    
    if (metric>0):
        label = 'POSITIVE'
        correct_answers+=1
    elif metric < 0:
        label = 'NEGATIVE'
    else:
        label = 'NEUTRAL'
    metrics.append(metric)
    predicted.append(label)
    print(f'Classified as {label} by metric: {metric}')
    
    

for query in all_negative_tweets[4000:]:
    print(f' {query} of class NEGATIVE:')
    queries.append(query)
    actual.append('NEGATIVE')
    
    q_rules = sc.parallelize([query]).flatMap(make_the_pairs) \
                    .map(lambda x: (x,1)) \
                    .reduceByKey(lambda a,b: a+b)
    q_rules = q_rules.collect()

    #metric = (+ -)(support)*confident
    metric = 0
    for rule in q_rules:
        if len(df[df['RULE'] == rule[0]])>0:
            row = df[df['RULE'] == rule[0]]
            support = row.SUPPORT.values[0]
            confidence = row.CONFIDENCE.values[0]
            coef = row.POS_COUNT.values[0] - row.NEG_COUNT.values[0]
            metric += coef * support * confidence
    
    if (metric>0):
        label = 'POSITIVE'
    elif metric < 0:
        label = 'NEGATIVE'
        correct_answers+=1
    else:
        label = 'NEUTRAL'
    metrics.append(metric)
    predicted.append(label)
    print(f'Classified as {label} by metric: {metric}')
    


accuracy = correct_answers / len(all_positive_tweets[4000:]+all_negative_tweets[4000:])
print(f'This model passed the testset by the accuracy of {accuracy}')
        
results = pd.DataFrame({'QUERY':queries, 'ACTUAL_CLASS':actual, 'PREDICTED_CLASS':predicted, 'METRIC':metrics})

 Bro:U wan cut hair anot,ur hair long Liao bo
Me:since ord liao,take it easy lor treat as save $ leave it longer :)
Bro:LOL Sibei xialan of class POSITIVE:


Classified as NEGATIVE by metric: -0.13396148198779775
 @heyclaireee is back! thnx God!!! i'm so happy :) of class POSITIVE:
Classified as NEUTRAL by metric: 0
 @BBCRadio3 thought it was my ears which were malfunctioning, thank goodness you cleared that one up with an apology :-) of class POSITIVE:
Classified as NEGATIVE by metric: -0.1148278745843631
 @HumayAG 'Stuck in the centre right with you. Clowns to the right, jokers to the left...' :) @orgasticpotency @ahmedshaheed @AhmedSaeedGahaa of class POSITIVE:
Classified as NEGATIVE by metric: -0.06108303986117491
 Happy Friday :-) http://t.co/iymPIlWXFY of class POSITIVE:
Classified as POSITIVE by metric: 10.241669690372522
 @Sazzi91 we are following you now :) x of class POSITIVE:
Classified as POSITIVE by metric: 0.0242494010095172
 My #TeenChoice For #ChoiceinternationalArtist is #SuperJunior Fighting Oppa :D of class POSITIVE:
Classified as POSITIVE by metric: 2.1586527777777773
 @FindBenNeedham it's my birthday today so for my bir

Classified as POSITIVE by metric: 0.07767298931476763
 @Real_Liam_Payne please follow me liam :) of class POSITIVE:
Classified as NEGATIVE by metric: -13.492308123406282
 @LaurakBuzz lovely write up :) I absolutely adore the NVA and am lucky enough to live 5 minuets away. Roll in GameCity! of class POSITIVE:
Classified as NEGATIVE by metric: -7.2795858108325024
 #FF @Brianscrooks Thanks Brian :) @GOVManslaughter @GlobaluxGenesis of class POSITIVE:
Classified as POSITIVE by metric: 0.049591541588492806
 @ElliotHorwoodF1 @LewisHamilton His headrest was loose, that's the reason he pitted :-) of class POSITIVE:
Classified as NEGATIVE by metric: -0.11062777929474177
 @batesm0t3l Hi there, I've spoken with the store who have advised they do have a PayPoint and that card can be topped up :) Thanks, Beth of class POSITIVE:
Classified as POSITIVE by metric: 23.184531947445592
 @BCTGA Will do! Thanks for your help :-) of class POSITIVE:
Classified as POSITIVE by metric: 67.1769171533473
 Deepthr

Classified as NEGATIVE by metric: -0.09785470359476872
 Morning! Hospital day today ! Hoping to finally get signed off after a long year of pain and ops ! #Fingerscrossed.:) of class POSITIVE:
Classified as POSITIVE by metric: 0.002988704204331053
 The best photobomb :D [pic] — https://t.co/3dP40WD5n2 of class POSITIVE:
Classified as POSITIVE by metric: 0.3276875700378204
 @mazt_t @Peter_Fitz @SMHsport thanks SE :) of class POSITIVE:
Classified as NEUTRAL by metric: 0
 Hello :) Get Youth Job Opportunities follow &gt;&gt; @tolajobjobs @KingsleyDali of class POSITIVE:
Classified as POSITIVE by metric: 53.166192659176524
 I just finished day 33 of the @YouVersion plan 'Bible in 90 Days'. Check it out here: 2 Chronicles 23 completed :) http://t.co/3Yw3TFCRk6 of class POSITIVE:
Classified as NEGATIVE by metric: -25.354322839068526
 @TMobile Yo gimme this month free on my bill since service has been out pretty much every other day :)))))))))))))))))))))))))) of class POSITIVE:
Classified as 

Classified as NEGATIVE by metric: -0.9709847773605151
 I just woke up and I feel incredibly sick idk if it's cause I drank Starbucks at 11 o'clock last night or if it's a reaction to my meds :) of class POSITIVE:
Classified as NEGATIVE by metric: -45.592502447106824
 @AndyCarnochan @WickermanFest Oops...that's called a #FridayFauxPas :) I'll get that changed right now! Thank you, G of class POSITIVE:
Classified as POSITIVE by metric: 0.0067972814407210955
 I will fulfil all your fantasies :) 👉 http://t.co/wKpgkiDEmP 👈💗 http://t.co/HHoyjasr1A of class POSITIVE:
Classified as POSITIVE by metric: 3.526397324332926
 calling it a night and going to sleep Now :) of class POSITIVE:
Classified as NEGATIVE by metric: -0.006331953460442555
 yeah they're giving me subtle hints. also tip #2: don't use it all and the community these last few years. :-) Prim’s Algorithm III of class POSITIVE:
Classified as NEUTRAL by metric: 0
 @JewelStaite Funny, but yes too soon. It's always going to be too soon. 

...

Classified as NEGATIVE by metric: -48.44171741919779
 They didn't care that I was a Muslim :( of class NEGATIVE:
Classified as NEGATIVE by metric: -33.26968460102746
 @sneaksontaylor no it's not :( of class NEGATIVE:
Classified as NEGATIVE by metric: -0.03796577793322393
 When Jessica calls and quits on power abs at 5:15 :-( of class NEGATIVE:
Classified as NEGATIVE by metric: -0.004240631163708087
 She lost us, her friends... :( she's the one who started the argument. of class NEGATIVE:
Classified as NEGATIVE by metric: -0.0004973207840524182
 @MsMeghanMakeup hope you're having fun at vidcon!! rlly bummed i couldnt go :(( love you sunshine 💛💛💛💛 of class NEGATIVE:
Classified as POSITIVE by metric: 0.5467236554086382
 @iTaimikhan yepp.. I'm getting bore :( of class NEGATIVE:
Classified as NEGATIVE by metric: -0.012584953374427058
 last night was so good :( 😺💒💎🎉 of class NEGATIVE:
Classified as POSITIVE by metric: 0.2825463084128334
 @PrinceOfRnbZJM Can you follow me ? :( of class NEGATI

Classified as NEUTRAL by metric: 0
 @eyeammarkk omg :( not gonna happen , yeah we wish.. of class NEGATIVE:
Classified as NEGATIVE by metric: -0.015404031154326633
 @parisianskies :( at lest you have the GP to get you through it ^_^ of class NEGATIVE:
Classified as NEGATIVE by metric: -0.18345394034658746
 @nambooti aww :( catch them at the departure then! of class NEGATIVE:
Classified as NEUTRAL by metric: 0
 @98_madhvi_tommo  it apparently isnt :( of class NEGATIVE:
Classified as NEUTRAL by metric: 0
 @SensodyneIndia i miss those days :( #ToothSensitivity. of class NEGATIVE:
Classified as NEUTRAL by metric: 0
 @natalieegiselle it's unavailable now :( of class NEGATIVE:
Classified as NEUTRAL by metric: 0
 @i_am_mill_i_am you mean we can't wipe them out? :-( of class NEGATIVE:
Classified as NEGATIVE by metric: -0.014725096549408111
 Just found out my flight home is 13 hours long :((((((( of class NEGATIVE:
Classified as NEGATIVE by metric: -0.16974258563074351
 @oIiverfelicitys UGH YUC

Classified as POSITIVE by metric: 0.3572916666666666
 @MaddyyOR @TreqEu i Wish But b8 Too Strong :( of class NEGATIVE:
Classified as NEGATIVE by metric: -0.01302677282804739
 Hey girl you must.be a dairy produxt because I want you but I can't have you I am lactose intolerant. :( of class NEGATIVE:
Classified as NEGATIVE by metric: -313.4311436206073
 @ITVCentral #Midlands Yes thanks for the depressing weather forecast, where the word 'rain' was mentioned several times :-( of class NEGATIVE:
Classified as NEGATIVE by metric: -0.2358539249598464
 Why am I crying?! :( of class NEGATIVE:
Classified as NEGATIVE by metric: -5.380905858052426
 @emergingrecruit @_RobLong You'll be this old one day..... :( of class NEGATIVE:
Classified as NEUTRAL by metric: 0
 literally spent all day yesterday sleeping on and off in bed, and I'm still absolutely fucking knackered today :(( of class NEGATIVE:
Classified as POSITIVE by metric: 0.005748002787476472
 this new Sandra Bland footage is really the icin

Classified as NEGATIVE by metric: -1.220232771596291
 I hit my thumb :(. of class NEGATIVE:
Classified as POSITIVE by metric: 0.03396704180064308
 @iloveksoo we almost got to see his cute ankles but his socks :( of class NEGATIVE:
Classified as NEGATIVE by metric: -0.8126694379782876
 @bravefrontiergl Honestly at this point I don't care about compensation. I just want to be able to play BF again. :( of class NEGATIVE:
Classified as NEGATIVE by metric: -114.17788733836335
 @dnwrld that song is not on here :( of class NEGATIVE:
Classified as NEGATIVE by metric: -0.010016345368011152
 @CourtneyR_Green awwwww my baby :( 💕 of class NEGATIVE:
Classified as NEUTRAL by metric: 0
 I couldn't sleep last night n it didn't help me waking up at 8 this morning n my phone wasn't even charged :( of class NEGATIVE:
Classified as NEGATIVE by metric: -10.58530980741074
 @ZozeeBo @TanyaBurr please can you tell everyone why you didnt go to vidcon :( of class NEGATIVE:
Classified as NEGATIVE by metric: -0.2

Classified as POSITIVE by metric: 0.28143661137440756
 ill be on soon, I PROMISE :(
waaah of class NEGATIVE:
Classified as NEGATIVE by metric: -11.179312022382764
 I wanna change my avi but uSanele :( of class NEGATIVE:
Classified as NEGATIVE by metric: -123.65270567302223
 MY PUPPY BROKE HER FOOT :( of class NEGATIVE:
Classified as NEGATIVE by metric: -0.00585455486542443
 where's all the jaebum baby pictures :(( of class NEGATIVE:
Classified as NEUTRAL by metric: 0
 But but Mr Ahmad Maslan cooks too :( https://t.co/ArCiD31Zv6 of class NEGATIVE:
Classified as NEUTRAL by metric: 0
 @eawoman As a Hull supporter I am expecting a misserable few weeks :-( of class NEGATIVE:
Classified as NEGATIVE by metric: -0.019610880765686994
This model passed the testset by the accuracy of 0.467


In [31]:
results

NameError: name 'results' is not defined

In [509]:
results.to_csv('results.csv')

In [34]:
results = pd.read_csv('results.csv')
results

,Unnamed: 0,QUERY,ACTUAL_CLASS,PREDICTED_CLASS,METRIC
0,0,"Bro:U wan cut hair anot,ur hair long Liao bo\n...",POSITIVE,NEGATIVE,-0.133961
1,1,@heyclaireee is back! thnx God!!! i'm so happy :),POSITIVE,NEUTRAL,0.000000
2,2,@BBCRadio3 thought it was my ears which were m...,POSITIVE,NEGATIVE,-0.114828
3,3,@HumayAG 'Stuck in the centre right with you. ...,POSITIVE,NEGATIVE,-0.061083
4,4,Happy Friday :-) http://t.co/iymPIlWXFY,POSITIVE,POSITIVE,10.241670
...,...,...,...,...,...
1996,1995,I wanna change my avi but uSanele :(,NEGATIVE,NEGATIVE,-123.652706
1997,1996,MY PUPPY BROKE HER FOOT :(,NEGATIVE,NEGATIVE,-0.005855
1998,1997,where's all the jaebum baby pictures :((,NEGATIVE,NEUTRAL,0.000000
1999,1998,But but Mr Ahmad Maslan cooks too :( https://t...,NEGATIVE,NEUTRAL,0.000000


In [514]:
accuracy

0.467

In [515]:
len(results)

2000

In [526]:
len(results[results['ACTUAL_CLASS']==results['PREDICTED_CLASS']])

934

In [525]:
len(results[results['PREDICTED_CLASS'] == 'NEUTRAL'])

621

In [528]:
explicit_accuracy = (934+621)/len(results)
explicit_accuracy

0.7775

In [24]:
results = pd.read_csv('results.csv').drop('Unnamed: 0',axis=1)
df = pd.read_csv('RULE_SC.csv').drop('Unnamed: 0',axis=1)
results

,QUERY,ACTUAL_CLASS,PREDICTED_CLASS,METRIC
0,"Bro:U wan cut hair anot,ur hair long Liao bo\n...",POSITIVE,NEGATIVE,-0.133961
1,@heyclaireee is back! thnx God!!! i'm so happy :),POSITIVE,NEUTRAL,0.000000
2,@BBCRadio3 thought it was my ears which were m...,POSITIVE,NEGATIVE,-0.114828
3,@HumayAG 'Stuck in the centre right with you. ...,POSITIVE,NEGATIVE,-0.061083
4,Happy Friday :-) http://t.co/iymPIlWXFY,POSITIVE,POSITIVE,10.241670
...,...,...,...,...
1996,I wanna change my avi but uSanele :(,NEGATIVE,NEGATIVE,-123.652706
1997,MY PUPPY BROKE HER FOOT :(,NEGATIVE,NEGATIVE,-0.005855
1998,where's all the jaebum baby pictures :((,NEGATIVE,NEUTRAL,0.000000
1999,But but Mr Ahmad Maslan cooks too :( https://t...,NEGATIVE,NEUTRAL,0.000000


In [38]:
neutrals = results[results['PREDICTED_CLASS'] == 'NEUTRAL']
len(neutrals)

621

In [64]:
for index, row in neutrals.iterrows():
    query = row.QUERY
    q_rules = sc.parallelize([query]).flatMap(make_the_pairs) \
                    .map(lambda x: (x,1)) \
                    .reduceByKey(lambda a,b: a+b)
    q_rules = q_rules.collect()
    metric = 0
    for rule in q_rules:
        if len(df[df['RULE'] == rule[0]])>0:
            print('Rule is present in our db')
            r_row = df[df['RULE'] == rule[0]]
            support = r_row.SUPPORT.values[0]
            confidence = r_row.CONFIDENCE.values[0]
            coef = r_row.POS_COUNT.values[0] - r_row.NEG_COUNT.values[0]
            if coef == 0:
                coef = 1
            prev_metric = metric
            print(f'calculating {coef}*{support}*{confidence}')
            metric += coef * support * confidence
            print(f'new metric{metric} and prev metic {prev_metric}')
            if metric == 0:
                metric = prev_metric
    new_row = results[results['QUERY'] == query]
    if (metric>0):
        label = 'POSITIVE'
        results.at[new_row.index[0],'PREDICTED_CLASS'] = 'POSITIVE'
    elif metric < 0:
        label = 'NEGATIVE'
        results.at[new_row.index[0],'PREDICTED_CLASS'] = 'NEGATIVE'
    else:
        label = 'NEUTRAL'
    print(f'Classified as {label} by metric: {metric}')

Classified as NEUTRAL by metric: 0
Rule is present in our db
calculating 1*0.025*0.0022222222222222222
new metric5.555555555555556e-05 and prev metic 0
Rule is present in our db
calculating 1*0.025*0.18181818181818182
new metric0.004601010101010101 and prev metic 5.555555555555556e-05
Classified as POSITIVE by metric: 0.004601010101010101
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metr

Rule is present in our db
calculating 1*0.125*0.03484320557491289
new metric0.06117358251504593 and prev metic 0.056818181818181816
Classified as POSITIVE by metric: 0.06117358251504593
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Rule is present in our db
calculating 1*0.02

Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRA

Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Classified as NEUTRAL by metric: 0
Rule is present in our db
calculating 1*0.025*0.0012861736334405145
new metric3.2154340836012867e-05 and prev metic 0
Rule is

In [65]:
new_neutrals = results[results['PREDICTED_CLASS'] == 'NEUTRAL']
len(new_neutrals)

603

In [68]:
correct_predicted = results[results['PREDICTED_CLASS'] == results['ACTUAL_CLASS']]
correct_predicted

,Unnamed: 0,QUERY,ACTUAL_CLASS,PREDICTED_CLASS,METRIC
4,4,Happy Friday :-) http://t.co/iymPIlWXFY,POSITIVE,POSITIVE,10.241670
5,5,@Sazzi91 we are following you now :) x,POSITIVE,POSITIVE,0.024249
6,6,My #TeenChoice For #ChoiceinternationalArtist ...,POSITIVE,POSITIVE,2.158653
9,9,@LouiseR97054900 Happy Friday for you too :) @...,POSITIVE,POSITIVE,11.201564
10,10,:3 ! Good nigth :): estoy escuchando a Enemy...,POSITIVE,POSITIVE,0.001289
...,...,...,...,...,...
1992,1991,@louanndavies Completely agree. The press won'...,NEGATIVE,NEGATIVE,-0.000375
1995,1994,"ill be on soon, I PROMISE :(\nwaaah",NEGATIVE,NEGATIVE,-11.179312
1996,1995,I wanna change my avi but uSanele :(,NEGATIVE,NEGATIVE,-123.652706
1997,1996,MY PUPPY BROKE HER FOOT :(,NEGATIVE,NEGATIVE,-0.005855


In [70]:
accuracy = len(correct_predicted)/len(results) * 100
accuracy

47.17641179410295